## Import Packages

In [1]:
import sys
sys.path.append('..')

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
import pandas as pd
import sys
import os

from sklearn.model_selection import train_test_split

from detector.data_loader import LoadEnronData, LoadPhishingData, LoadSocEnggData
from detector.labeler import EnronLabeler, MismatchLabeler
from ethics.differential_privacy import RandomForestPrivacyModel
from sklearn.ensemble import RandomForestClassifier
from detector.preprocessor import Preprocessor
from utils.util_modeler import evaluate_and_log, get_f1_score, Augmentor

import wandb
import argparse

from sklearn.pipeline import Pipeline
from diffprivlib.models.logistic_regression import LogisticRegression


[nltk_data] Downloading package stopwords to
[nltk_data]     /common/home/ps1279/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Init wandb for model tracking

In [2]:
wandbdict = {
    'key': os.getenv('WANDB_API_KEY'),
    'entity': os.getenv('WANDB_ENTITY'),
    'project': os.getenv('WANDB_PROJECT'),
}
wandb.login(key=wandbdict['key'])
run = wandb.init(project=wandbdict['project'], entity=wandbdict['entity'])

12/06/2023 11:17:49:ERROR:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: advaithrao (regressors). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /common/home/ps1279/.netrc


## Load Data

In [3]:
#The data file is too large to upload to github, so you will need to run from https://github.com/advaithsrao/Fraud-Detector/wiki/Load-Preprocessed-and-Labeled-Data#The data file is too large to upload to github, 
#so you will need to run data loading from https://github.com/advaithsrao/Fraud-Detector/wiki/Load-Preprocessed-and-Labeled-Data 
#and save it to <repo>/data/fraud_detector_data.csv
data = pd.read_csv('./data/fraud_detector_data.csv')

## Data Splits

In [4]:
train_data = data[data.Split == 'Train']
sanity_data = data[data.Split == 'Sanity']
gold_fraud_data = data[data.Split == 'Gold Fraud']

## Run Augmentation

In [12]:

augmentor = Augmentor()

train_body, train_labels = augmentor(
    train_data['Body'].tolist(),
    train_data['Label'].tolist(),
    aug_label=1,
    num_aug_per_label_1=9,
    shuffle=True
)

train_data = pd.DataFrame(
    {
        'Body': train_body,
        'Label': train_labels
    }
)

train_data.drop_duplicates(subset=['Body'], inplace=True)
train_data.reset_index(drop=True, inplace=True)


In [66]:
# train_data.to_csv('./data/augmented_train_data.csv', index=None)

## Train Model

In [6]:
# train_data = pd.read_csv('./data/augmented_train_data.csv')

In [7]:
body = train_data['Body'].tolist()
label = train_data['Label'].tolist()

In [8]:
body_train, body_val, label_train, label_val = train_test_split(body, label, test_size=0.2, random_state=42, stratify=label)

In [9]:
from utils.util_modeler import Word2VecEmbedder
import matplotlib.pyplot as plt
import numpy as np

vectorizer = Word2VecEmbedder()
# Call your code that produces output
# model.train(train_data['Body'], train_data['Label'], wandb = run)

epsilons = [1e-8, 1e-2, 1, 7.5, 20]
accuracies = []

for eps in epsilons:
    model = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', LogisticRegression(n_jobs=-1, epsilon = eps))
    ])

    model.fit(body_train, label_train)

    accuracy = get_f1_score(label_val, model.predict(body_val), average = 'macro')
    accuracies.append(accuracy)


plt.plot(epsilons, accuracies, marker='o')
plt.xscale('log')  # Use a logarithmic scale for better visibility
plt.xlabel('Epsilon')
plt.ylabel('F1 Score')
plt.title('F1 Score vs Epsilon')
plt.grid(True)

plt.savefig("lr_dp_accuracy_vs_epsilon_plot.png")

# Log the plot to wandb
wandb.log({"Accuracy vs Epsilon": plt})

plt.show()

print(f'{"="*20} \n Best Model for Epsilon = {epsilons[np.argmax(accuracies)]} with Validation F1 Score = {np.max(accuracies)} \n {"="*20}')

#Fit model with best epsilon
model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', LogisticRegression(n_jobs=-1, epsilon = epsilons[np.argmax(accuracies)]))
])

model.fit(body_train, label_train)
 
print(f'{"="*20} Training Done {"="*20}')


/common/home/ps1279/.cache/pypoetry/virtualenvs/ethical-fraud-detector-qnRNkJHZ-py3.10/lib/python3.10/site-packages/diffprivlib/models/logistic_regression.py:239: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "
/common/home/ps1279/.cache/pypoetry/virtualenvs/ethical-fraud-detector-qnRNkJHZ-py3.10/lib/python3.10/site-packages/diffprivlib/models/logistic_regression.py:239: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be 

 Best Model for Epsilon = 20 with Validation F1 Score = 0.9192553965725826 


/common/home/ps1279/.cache/pypoetry/virtualenvs/ethical-fraud-detector-qnRNkJHZ-py3.10/lib/python3.10/site-packages/diffprivlib/models/logistic_regression.py:239: PrivacyLeakWarning:

Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.



==================== Training Done ====================


/common/home/ps1279/.cache/pypoetry/virtualenvs/ethical-fraud-detector-qnRNkJHZ-py3.10/lib/python3.10/site-packages/diffprivlib/models/logistic_regression.py:419: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  warnings.warn("lbfgs failed to converge. Increase the number of iterations.", ConvergenceWarning)


In [10]:
f1_scores = {}
os.makedirs('/common/home/ps1279/models/rf_diff_privacy/2023-12-05/rf_diff_privacy/logs', exist_ok=True)
save_path='/common/home/ps1279/models/rf_diff_privacy/2023-12-05/rf_diff_privacy/'

## Predict on all datasets and generate logs + mismatch_data

In [11]:
train_data['Prediction'] = model.predict(train_data['Body'])

preds = train_data['Prediction'].tolist()

evaluate_and_log(x=train_data['Body'].tolist(), y_true=train_data['Label'].tolist(), y_pred=train_data['Prediction'].tolist(), filename=os.path.join(save_path,'logs/train.log'), experiment=run)#, id = train_data['Mail-ID'].tolist())
f1_scores['train'] = get_f1_score(y_true=train_data['Label'].tolist(), y_pred=train_data['Prediction'].tolist())

In [12]:
sanity_data['Prediction'] = model.predict(sanity_data['Body'])

evaluate_and_log(x=sanity_data['Body'].tolist(), y_true=sanity_data['Label'].tolist(), y_pred=sanity_data['Prediction'].tolist(), filename=os.path.join(save_path,'logs/sanity.log'), experiment=run)#, id = sanity_data['Mail-ID'].tolist())
f1_scores['sanity'] = get_f1_score(y_true=sanity_data['Label'].tolist(), y_pred=sanity_data['Prediction'].tolist())



wandb: WARNING Serializing object of type str that is 103105 bytes
wandb: WARNING Serializing object of type str that is 123159 bytes
wandb: WARNING Serializing object of type str that is 278404 bytes
wandb: WARNING Serializing object of type str that is 1401178 bytes


In [13]:
gold_fraud_data['Prediction'] = model.predict(gold_fraud_data['Body'])

evaluate_and_log(x=gold_fraud_data['Body'].tolist(), y_true=gold_fraud_data['Label'].tolist(), y_pred=gold_fraud_data['Prediction'].tolist(), filename=os.path.join(save_path,'logs/gold_fraud.log'), experiment=run)#, id = gold_fraud_data['Mail-ID'].tolist())
f1_scores['gold_fraud'] = get_f1_score(y_true=gold_fraud_data['Label'].tolist(), y_pred=gold_fraud_data['Prediction'].tolist())



In [14]:
f1_scores

{'train': 0.9180848923103692,
 'sanity': 0.9737548053931238,
 'gold_fraud': 0.9604989604989606}

In [15]:
#save mismatch data into a csv file
mismatch_data = pd.concat(
    [
        train_data[train_data['Prediction'] != train_data['Label']],
        sanity_data[sanity_data['Prediction'] != sanity_data['Label']],
        gold_fraud_data[gold_fraud_data['Prediction'] != gold_fraud_data['Label']]
    ],
    axis=0,
    ignore_index=True
)

mismatch_data.to_csv(os.path.join(save_path,'logs/mismatch_data.csv'), index=False)

## Save Logs

In [16]:
all_params = {**hyper_params, **f1_scores}
run.config.update(all_params)

In [17]:
logs_path = os.path.join(save_path,'logs')
log_artifact = wandb.Artifact("fraud-detector-logs", type="logs")
log_artifact.add_dir(logs_path)
run.use_artifact(log_artifact)

wandb: Adding directory to artifact (/common/home/ps1279/models/rf_diff_privacy/2023-12-05/rf_diff_privacy/logs)... Done. 0.3s


<Artifact QXJ0aWZhY3Q6NjU5MjgwMDc5>

## Save Model

In [20]:
from mlflow.sklearn import save_model
save_model(model, os.path.join(save_path,'model'))

In [21]:
model_path = os.path.join(save_path, 'model')
model_artifact = wandb.Artifact("fraud-detector-model", type="model")
model_artifact.add_dir(model_path)
run.use_artifact(model_artifact)

wandb: Adding directory to artifact (/common/home/ps1279/models/rf_diff_privacy/2023-12-05/rf_diff_privacy/model)... Done. 12.1s


<Artifact QXJ0aWZhY3Q6NjU5Mjg0NzIz>

In [22]:
# !rm -rf /common/home/ps1279/models/rf_diff_privacy/

In [23]:
run.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
